In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
# SQLite3 버전 문제 해결: Chroma는 SQLite3 3.35.0 이상이 필요하지만 시스템에는 낮은 버전이 설치되어 있음
# pysqlite3-binary를 사용하여 더 높은 버전의 SQLite3을 제공
import sys
import pysqlite3

# 기존 sqlite3 모듈을 pysqlite3으로 대체
sys.modules['sqlite3'] = pysqlite3
print(f"Using SQLite version: {pysqlite3.sqlite_version}")

Using SQLite version: 3.46.1


In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import chromadb

client = chromadb.PersistentClient('./real_estate_tax_collection')

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')

vector_store = Chroma(
    client=client,
    embedding_function=embedding_function,
    collection_name='real_estate_tax'
)

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [6]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query : str # 사용자 질문
    answer : str # 세율
    tax_base_equation : str # 과세표준 계산 수식
    tax_deduction : str # 공제액
    market_ratio : str # 공정시장가액비율
    tax_base : str # 과세표준 계산

graph_builder = StateGraph(AgentState)

In [7]:
query = '5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?'

In [17]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o")
small_llm = ChatOpenAI(model="gpt-4o-mini")

rag_prompt = hub.pull("rlm/rag-prompt")

In [20]:
# 동적으로 context는 retriever로부터 가져오고 질문은 RunnablePassthrough()를 사용해 동적으로 질문을 받는다.
tax_base_retrieval_chain = {'context' : retriever, 'question' : RunnablePassthrough()} | rag_prompt | llm | StrOutputParser()

tax_base_equation_promt = ChatPromptTemplate.from_messages([
    ('system', '사용자의 질문에서 과세표준을 계산하는 방법을 수식으로 나타내주세요. 부연 설명 없이 수식만 리턴해주세요'),
    ('human', '{tax_base_equation_information}')
])

tax_base_equation_chain = (
    {'tax_base_equation_information' : RunnablePassthrough()} | tax_base_equation_promt | llm | StrOutputParser()
)

tax_base_chain = {'tax_base_equation_information' : tax_base_retrieval_chain} | tax_base_equation_chain

def get_tax_base_equation(state: AgentState) :
    tax_base_equation_question = '주택에 대한 종합부동산세 계산시 과세표준을 계산하는 방법을 수식으로 표현해서 알려주세요'
    tax_base_equation = tax_base_equation_chain.invoke(tax_base_equation_question)
    return {'tax_base_equation' : tax_base_equation}

In [21]:
get_tax_base_equation({})

# {'tax_base_equation': '과세표준 = 공시가격 × 공정시장가액 비율 - 기본공제액'}

{'tax_base_equation': '과세표준 = 공시가격 × 공정시장가액 비율 - 기본공제액'}

In [ ]:
tax_deduction_chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

def get_tax_deduction(state: AgentState) :
    tax_deduction_question = '주택에 대한 종합부동산세 계산시 공제금액을 알려주세요'
    tax_deduction = tax_deduction_chain.invoke(tax_deduction_question)
    return {'tax_deduction' : tax_deduction}